In [17]:
#Importación librelias
import copy
import math
import networkx as nx
import numpy as np 
from collections import defaultdict
import itertools


#Variables

#Variable de bases de datos

radios_cov = [] # Radios covalentes(Encontrados en Wikipedia)
simbolos_cov = [] # Símbolos de átomos asociados a los radios

#Variables para generar la molecula

simbolos_molecula = []
molecula = nx.Graph()
vertices = []
coordx = []
coordy = []
coordz = []

#Variables para matriz de adyacencia

ma_ad= []

#Calculo de cargas

matriz_pos_val = []
uis = []
BO_final = []
valencias_finales = []
cargas_finales = []
# Imprimir

atomos_mol2 = []
enlaces_mol2 = []
#Lista de valencias y atomos de valencia 

global __ATOM_LIST__
__ATOM_LIST__ = \
    ['h',  'he',
     'li', 'be', 'b',  'c',  'n',  'o',  'f',  'ne',
     'na', 'mg', 'al', 'si', 'p',  's',  'cl', 'ar',
     'k',  'ca', 'sc', 'ti', 'v ', 'cr', 'mn', 'fe', 'co', 'ni', 'cu',
     'zn', 'ga', 'ge', 'as', 'se', 'br', 'kr',
     'rb', 'sr', 'y',  'zr', 'nb', 'mo', 'tc', 'ru', 'rh', 'pd', 'ag',
     'cd', 'in', 'sn', 'sb', 'te', 'i',  'xe',
     'cs', 'ba', 'la', 'ce', 'pr', 'nd', 'pm', 'sm', 'eu', 'gd', 'tb', 'dy',
     'ho', 'er', 'tm', 'yb', 'lu', 'hf', 'ta', 'w',  're', 'os', 'ir', 'pt',
     'au', 'hg', 'tl', 'pb', 'bi', 'po', 'at', 'rn',
     'fr', 'ra', 'ac', 'th', 'pa', 'u',  'np', 'pu']


global atomic_valence

atomic_valence = defaultdict(list)
atomic_valence[1] = [1]
atomic_valence[5] = [3,4]
atomic_valence[6] = [4]
atomic_valence[7] = [3,4]
atomic_valence[8] = [2,1,3]
atomic_valence[9] = [1]
atomic_valence[14] = [4]
atomic_valence[15] = [5,3] #[5,4,3]
atomic_valence[16] = [6,3,2] #[6,4,2]
atomic_valence[17] = [1]
atomic_valence[32] = [4]
atomic_valence[35] = [1]
atomic_valence[53] = [1]

global atomic_valence_electrons

atomic_valence_electrons = {}
atomic_valence_electrons[1] = 1
atomic_valence_electrons[5] = 3
atomic_valence_electrons[6] = 4
atomic_valence_electrons[7] = 5
atomic_valence_electrons[8] = 6
atomic_valence_electrons[9] = 7
atomic_valence_electrons[14] = 4
atomic_valence_electrons[15] = 5
atomic_valence_electrons[16] = 6
atomic_valence_electrons[17] = 7
atomic_valence_electrons[32] = 4
atomic_valence_electrons[35] = 7
atomic_valence_electrons[53] = 7


with open('Radios_Covalentes.txt') as rad: #Lee documento de radios covalentes
    lineas=rad.readlines()
rad.close()
for i in range (len(lineas)):
    k=lineas[i].split()
    simbolos_cov.append(k[0]) 
    radios_cov.append(float(k[1]))

def simbolo_atomo(atom): #convert integer atom to string atom

    global __ATOM_LIST__
    atom = __ATOM_LIST__[atom - 1]
    
    return atom


def entero_atomo(atom): # convert str atom to integer atom
    global __ATOM_LIST__
    atom = atom.lower()
    
    return __ATOM_LIST__.index(atom) + 1
    
def lee_xyz(nombre):#función que lee documento .xyz 
    nodo_sh=0  
    nombre_xyz=nombre+'.xyz'
    with open(nombre_xyz) as xyz:  #Abre archivo xyz
        lineas = xyz.readlines()
        n=int(lineas[0])
    for i in range (2,2+n): #Lee las coordenadas y atomos
        k=lineas[i].split()
        simbolos_molecula.append(k[0]) #Simbolo de cada átomo
        molecula.add_node(len(coordx))
        vertices.append(len(coordx)) #Número de etiqueta del vertice
        coordx.append(float(k[1]))
        coordy.append(float(k[2]))
        coordz.append(float(k[3]))
    xyz.close()

def distancia_R3(n,m,x,y,z): #Función de distancia en R3
    d = math.sqrt((x[n]-x[m])**2+(y[n]-y[m])**2+(z[n]-z[m])**2)
    
    return d

def numero_de_enlaces(n):
    conect=0
    for j in range(len(vertices)):
        conect=conect+ma_ad[n][j]
        
    return conect

def id_radio_cov(mol):#Función que asocia un radio covalente a un símbolo 
    m=0
    for i in range(len(simbolos_cov)):
        if(simbolos_cov[i]==mol):
            m=i
            break
            
    return radios_cov[m]



def valencias_buenas(BO,valencias):

    numero_enlaces = np.array(BO).sum(axis=1)
    for i in range(len(valencias)):
        if numero_enlaces[i] > valencias[i]:
            return False

    return True

def crea_enlaces(UA, AC):
    
    enlaces=[]

    for k,i in enumerate(UA):
        for j in UA[k + 1:]:
            if AC[i][j] == 1:
                enlaces.append(tuple(sorted([i, j])))

    return enlaces

def obten_pares_UA(UA,m_ad):
    
    enlaces=crea_enlaces(UA,m_ad)
    
    if len(enlaces) == 0:
        return[()]
    
    G = nx.Graph()
    G.add_edges_from(enlaces)
    UA_pares = [list(nx.max_weight_matching(G))]
    return UA_pares
    
def calcula_carga_atomica(atomo,no_enlaces):
    
    if atomo == 1:
        carga = 1 - no_enlaces
    elif atomo == 5:
        carga = 3 - no_enlaces
    elif atomo == 15 and no_enlaces == 5:
        carga = 0
    elif atomo == 16 and no_enlaces == 6:
        carga = 0
    else:
        carga = atomic_valence_electrons[atomo] - 8 + no_enlaces

    return carga

def carga_buena(BO,AC,du,ua,ver,sim_mol):
    # Carga total
    Q = 0
    # Carga fragmentos
    q_lista = []
    no_enlaces = list(np.array(BO).sum(axis=1))
    for i in (ver):
        sim_atomo=simbolos_molecula[i]
        atomo=entero_atomo(sim_atomo)
        q = calcula_carga_atomica(atomo,no_enlaces[i])
        Q = Q+q
        if atomo==6:
            numero_enlaces_unicos_C = list(np.array(BO)[i, :]).count(1)
            if numero_enlaces_unicos_C == 2 and no_enlaces[i] == 2:
                Q = Q + 1
                q = 2
            if numero_enlaces_unicos_C == 3 and Q + 1 < 0: #Cambiar por CARGA
                Q = Q + 2
                q = 1                
        if q != 0:
            q_lista.append([i,q])

    return (0 == Q) #Cambiar por CARGA

def BO_buena(BO,AC,du,ua,ver,sim_mol,valencia,carga):
    
    
    verifica_valencias=valencias_buenas(np.array(BO),valencia)
    if not verifica_valencias:
        return False
    
    verifica_suma = (np.array(BO) - np.array(AC)).sum()==sum(du)
    verifica_carga = carga_buena(np.array(BO),np.array(AC),du,ua,ver,sim_mol)
    
    if verifica_suma and verifica_carga:
        return True
    
    return False
        

def calcula_ui(fila,ver,vale):
    
    enlaces=numero_de_enlaces(fila)
    ui=vale[fila]-enlaces
    
    return ui


def crea_matriz_vs(ver,sim_mol,m_ad):
    
    posibles_valencias=[]
    for i in ver:
        conect=numero_de_enlaces(i)
        num=entero_atomo(sim_mol[i])
        pos_val_atom=[]
        for j in atomic_valence[num]:
            if(conect<=j):
                pos_val_atom.append(j)
        if(pos_val_atom==[]):
            print("Error en valencias \n")
            break
        else:
            posibles_valencias.append(pos_val_atom)
    val_np=np.array(list(itertools.product(*posibles_valencias)))
    matriz_vs=val_np.tolist()
    return matriz_vs       

def calcular_DU_UA(vale,ver,sim_mol,m_ad):
    ua=[]
    du=[]
    carga=0
    for i in ver:
        k=calcula_ui(i,ver,vale)
        carga=carga+k
        if(k>0):
            ua.append(i)
            du.append(k)
    
    return ua, du

def construye_BO(ac, ua, du, valencia, par,ver,sim_mol):

    BO = ac.copy()
    DU_guardar = []

    while DU_guardar != du :
        
        for i, j in par:
            BO[i][j] += 1
            BO[j][i] += 1

        BO_valencias = list(np.array(BO).sum(axis=1))
        DU_guardar = copy.copy(du)
        ua, du = calcular_DU_UA(valencia,ver,sim_mol,BO_valencias)
        par = obten_pares_UA(ua, ac)[0]
        
    return BO

def crea_BO(ver,sim_mol,m_ad,carga):
    
    matriz_pos_val = crea_matriz_vs(ver,sim_mol,m_ad)
    mejor_BO = m_ad.copy()
    mejor_valencia = matriz_pos_val[0]
    mejor_carga =[0]*len(ver)
    for valencias in matriz_pos_val:
        UA,DU = calcular_DU_UA(valencias,ver,sim_mol,m_ad)
        if(len(UA)==0):
            verifica_bo=BO_buena(m_ad,m_ad,DU,UA,ver,sim_mol,valencias,carga) #AGREGAR CARGA AQUÍ
            if verifica_bo:
                mejor_valencia = valencias
                return m_ad, mejor_valencia
            
        verifica_bo = None
        verifica_carga = None

        pares_UA=obten_pares_UA(UA,m_ad)
        for pares in pares_UA:
            BO_posible=construye_BO(m_ad,UA,DU,valencias,pares,ver,sim_mol)
            verifica_bo=BO_buena(BO_posible,m_ad,DU,UA,ver,sim_mol,valencias,carga)
            verifica_carga=carga_buena(BO_posible,m_ad,DU,UA,ver,sim_mol)
            verifica_valencias= valencias_buenas(BO_posible,valencias)
            if verifica_bo and verifica_valencias:
                mejor_valencia = valencias
                return BO_posible, mejor_valencia
            elif np.array(BO_posible).sum()>=np.array(mejor_BO).sum() and verifica_carga and verifica_valencias:
                mejor_BO=BO_posible.copy()
                mejor_valencia = valencias
    return mejor_BO, mejor_valencia, mejor_carga

def crea_MA(mol,ver,x,y,z,sim_mol): #Función que crea enlaces y matriz de adjacencia
    for i in range(len(vertices)): #Creación de enlaces 
        r1=id_radio_cov(sim_mol[i])
        for j in range(i+1,len(vertices)):
            r2=id_radio_cov(sim_mol[j])
            dis=distancia_R3(i,j,x,y,z)
            if(dis<=r1+r2+0.3): #Tolerancia de 0.3, valor arb.
                mol.add_edge(i, j)
    matriz_np = nx.to_numpy_matrix(molecula, nodelist=vertices) #Crea matriz de adjacencia
    
    return matriz_np.tolist()

def crea_enlaces_print():
    enlaces=[]
    for i in vertices:
        for j in range(i+1,len(vertices)):
            if BO_final[i][j]!=0:
                k=[i,j,int(BO_final[i][j])]
                enlaces.append(k)
    
    return enlaces

def detecta_ciclos(ciclos):
    
    ciclos=nx.cycle_basis(molecula, 0)
    
    return ciclos  

def colinealidad_buena(ciclo):

    x1 = coordx[ciclo[0]]
    x2 = coordx[ciclo[1]]
    x3 = coordx[ciclo[2]]
    y1 = coordy[ciclo[0]]
    y2 = coordy[ciclo[1]]
    y3 = coordy[ciclo[2]]
    z1 = coordz[ciclo[0]]
    z2 = coordz[ciclo[1]]
    z3 = coordz[ciclo[2]]

    for i in range(3,len(ciclo)):
        a1 = coordx[ciclo[i]]-x1
        a2 = coordy[ciclo[i]]-y1
        a3 = coordz[ciclo[i]]-z1
        b1 = x2-x1
        b2 = y2-y1
        b3 = z2-z1
        c1 = x3-x1
        c2 = y3-y1
        c3 = z3-z1
        M = np.array([[a1, a2, a3], [b1, b2, b3], [c1, c2 ,c3]])
        det = np.linalg.det(M)
        if(0.5<abs(det)):
            
            return False
            
    return True
    
def pares_buenos(ciclo):
        
    e_pi=0
    for atomo in ciclo:
        entero = entero_atomo(simbolos_molecula[atomo])
        coordinaciones = molecula.degree(atomo)
        eletrones_valencia = atomic_valence_electrons[entero]
        valencia = valencias_finales[atomo]
        pi = valencia-coordinaciones-eletrones_valencia
        e_pi+=pi
    if(e_pi%4==2):
        return True
    return False

def ciclos_mol2(atomos_p,enlaces_p,ciclos_ar):
    
    for ciclo in ciclos_ar:
        for atomo in ciclo:
            sintaxis = atomos_p[atomo]+".ar"
            atomos_p[atomo]=sintaxis
            for enlace in range(len(enlaces_p)):
                if(enlaces_p[enlace][0]==atomo or enlaces_p[enlace][1]==atomo):
                    if(enlaces_p[enlace][0]==atomo):
                        for atomo2 in ciclo:
                            if (enlaces_p[enlace][1]==atomo2):
                                enlaces_p[enlace][2] = 'ar'
                                break
                    if(enlaces_p[enlace][1]==atomo):
                        for atomo2 in ciclo:
                            if (enlaces_p[enlace][2]==atomo2):
                                enlaces_p[enlace][2] = 'ar'

    return atomos_p,enlaces_p

def identifica_ciclos_ar(atomos_p,enlaces_p):
    
    ciclos = []
    ciclos_ar = []
    ciclos = detecta_ciclos(ciclos)
    for ciclo in ciclos:
        verifica_colinealidad = colinealidad_buena(ciclo)
        verficia_pares = pares_buenos(ciclo)
        if verifica_colinealidad and verficia_pares:
            ciclos_ar.append(ciclo)
    atomos_p,enlaces_p = ciclos_mol2(atomos_p,enlaces_p, ciclos_ar)
    
    return atomos_p,enlaces_p

def crea_archivo_mol(nombre,atomos_p,enlaces_p): # Función que crea del documento .mol2  
    atomos_p = simbolos_molecula.copy()
    enlaces_p = crea_enlaces_print()
    atomos_p,enlaces_p = identifica_ciclos_ar(atomos_p,enlaces_p)
    nombre_xyz=nombre+'.mol2'
    f = open(nombre_xyz, "w") 
    f.write("@<TRIPOS>MOLECULE\n")
    f.write(" Molden generated mol2\n")
    n1=len(simbolos_molecula)
    n2=len(enlaces_p)
    f.write("    %d     %d      1\n"%(n1,n2))
    f.write(" SMALL\n")
    f.write(" NO_CHARGES\n")
    f.write(" ****\n")
    f.write(" ****\n")
    f.write("@<TRIPOS>ATOM\n")
    for i in range (len(simbolos_molecula)): #Escribe información de los átomos de las moléculas
        f.write("{:>6}  {:>1}    {: .4f}    {: .4f}    {: .4f}  {:<3}   {:}    {:}\n".format(i+1,simbolos_molecula[i],coordx[i],coordy[i],coordz[i],atomos_p[i],"1 RES1","0.0000"))
    f.write("@<TRIPOS>BOND\n")
    for i in range (len(enlaces_p)): #Escribe información de los enlaces de las moléculas
        f.write("{:>6}{:>6}{:>6}{:>7}\n".format(i+1,enlaces_p[i][0]+1,enlaces_p[i][1]+1,enlaces_p[i][2])) 
    f.write("@<TRIPOS>SUBSTRUCTURE\n")
    f.write("      1 RES1       1\n")
    f.close()
    
nombre_arc = input("Escrible el nombre del archivo:")
carga = input("carga total:")
lee_xyz(nombre_arc) #Lee archivo xyz
ma_ad=crea_MA(molecula,vertices,coordx,coordy,coordz,simbolos_molecula) #Crea enlaces y matriz de adjacencia
BO_final,valencias_finales, cargas_finales = crea_BO(vertices,simbolos_molecula,ma_ad, carga)
crea_archivo_mol(nombre_arc,atomos_mol2,enlaces_mol2)


Escrible el nombre del archivo:prueba4
carga total:0
